#  Описание работы

## Задания
* Придумать, что делать с пропущенными отсчётами:
    1. [Теорема Котельникова](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%9A%D0%BE%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2%D0%B0)
* Оптимизировать работу кода, задейстовав все возможные потоки процессора - [параллельное программирование](https://www.youtube.com/watch?v=fKl2JW_qrso&list=RDCMUCCezIgC97PvUuR4_gbFUs5g&start_radio=1&rv=fKl2JW_qrso&t=28)
* Проверить корректность работы ЕМ-алгоритма:
    * В окне довожу ЕМ до точности $\varepsilon$. Не сдвигая окно запускаю ЕМ ещё раз, но с новыми начальными приближениями. Так делаю 10 раз, сохраняя информацию о нач. прибл-ях. Добавляю 11-ым набором предпоследние, 10-ые, начальные приближения с последнего окна и выбираю тот набор, который максимизирует функцию правдоподобия. Результат - более точная (уверенная) максимизация функции правдоподобия (более точное разделение смесей).   
* Построить 3D горы для смесей.

1. Первая стадия 
    * Данные по одной координате разделить на смеси в окне, считая что имеется __6__ нормальных законов. Суммарно получается 18-ть дополнительных массивов: мат. ож-е, ср. кв. откл. и веса для каждого закона.
    * Визуализировать полученные массивы (придумать как). Можно через составляющие валатильности и тренд.
    * В каждом окне считается энтропия $H = -\sum_{i=1}^{6} p_i \cdot \log{p_i}$
    
2. Далее разбиваемся на группы: построение многомерного ЕМ и изучения всплесков, их вер-ти.

## Информация от физиков

### Письмо

Пересылаю запрошенный в конце прошедшей недели набор файлов за 2023 год, включающий:
именования столбцов регистрируемых параметров + трехмесячные файлы изображения временных разверток + листинги этих разверток. __Период опроса__ -- 1 минута. Предобработка по ограничению уровня и скорости вариации индукции магнитиного поля. В сумме -- около 2.000.000 выборок. Кажущиеся дефекты с октября 2023 года соответствуют процессу предфильтрации, через месяц они исчезнут. Формат содержит только актуальные данные, необходимые для реконструкции.

Для проверочных кодов будем использовать четыре __синтетических__ параметра: SYM_H, ASYM_H, SYM_D, ASYM_D.







## Замечания






* Один и тот же файл выдают за разный - `omni_min_def_20230401_20230630.lst` и `omni_min_def_20230101_20230331.lst`. Это данные за Январь-Март-2023 и Апрель-Июнь-2023. Судя по всему потеряна именно информация со второго периода, так как значения переменной 'Day'.
* В данных за октябрь полностью отсутсвуют синтетические параметры. Да и в принципе мало данных за этот месяц.
* EM-алгоритм выдает пропуски на графиках в связи с тем, что на каком-то номере итерации среднеквадрат-е откл-е $\sigma$ становится равным нулю. Что превращает веса с индексами соответсвующими $\sigma_k=0$ на следующей итерации в неопределённые значения `nan`

### Памятка


* Не забывай при существенном изменении кода создавать новые ветки и работать на них.
* https://numpy.org/devdocs/user/basics.types.html - number types.
* https://plotly.com/python/3d-charts/ - plotly 3D
* https://plotly.github.io/plotly.py-docs/generated/plotly.express.histogram.html - plotly histogram
* https://plotly.com/python/reference/layout/#layout-title - plotly fig layout
* https://coolors.co/092327-0b5351-00a9a5-4e8098-90c2e7 - генератор политры цвета.

# Загрузка данных

In [2]:
import pandas as pd
import numpy as np
import os

import histplot as hp # Персональный модуль визуализации
import essentials as es # Персональный модуль основных алгоритмов


# Позволяет использовать измененные модули без перезагрузки ядра
%load_ext autoreload
%autoreload 2

# Позволяет выводит графики в ячейках
%matplotlib inline

# Путь к папке с данными
PATH_DATA = os.getcwd() + '/Data/'

# Указание типов для упрощения считывания данных из *.csv файлов
TYPES = dtypes = {
    'Year': int, # str на самом деле, просто с int удобнее работать
    'Day': int, # str
    'Hour': int, # str
    'Minute': int, # str
    'BX': float,
    'BY': float,
    'BZ': float,
    'Vx_Velocity': float,
    'Vy_Velocity': float,
    'Vz_Velocity': float,
    'SYM/D': int,
    'SYM/H': int,
    'ASY/D': int,
    'ASY/H': int }

# os.listdir(PATH_DATA) # выводит содержимое папки, лежащей по пути 'PATH_DATA'

# Считываени данных
# df1 = pd.read_csv(PATH_DATA + "january_march.csv", dtype=TYPES)
# df2 = pd.read_csv(PATH_DATA + "april_jun.csv", dtype=TYPES)
# df3 = pd.read_csv(PATH_DATA + "july_september.csv", dtype=TYPES)
# df4 = pd.read_csv(PATH_DATA + "october.csv")
# df = pd.read_csv(PATH_DATA + "FULL_DATA.csv")

data_clean = pd.read_csv(PATH_DATA + "dropna_january_march.csv", dtype=TYPES)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from converter import time_related_id
time_related_id(data_clean)

# Обработка периода Январь-Март

В связи упомянутым замечанием про отсутсвие данных с периода Февраль-Июнь и построенным графиком всех данных, анализ данных будет осуществляться на подвыборке Январь-Март

## Визуализация данных

### Временные ряды и их гистограммы

In [ ]:
custom_title_drop = "Пропуски \"склеены\"."
custom_xaxis = data_clean['ydhm_id']
hp.show_genral_info(data_clean['BX'], add_title=custom_title_drop, add_xaxis=custom_xaxis)

In [ ]:
data_clean

Комментарии:
1. Склеенные данные не выглядят сплошными из-за равномерности сетки по времени. На самом деле в "дырках" отсчетов нет

### 3D гистограммы временных рядов

In [ ]:
clean_hist_3D = es.construct_hist3D(data_clean['BX'], window_size=4000, step=100)

hp.movable3D_hist(clean_hist_3D)

clean_BX_3D_0_3000_30 = es.construct_hist3D(data_clean['BX'][:3000], step=30)
clean_BY_3D_0_3000_30 = es.construct_hist3D(data_clean['BY'][:3000], step=30)
clean_BZ_3D_0_3000_30 = es.construct_hist3D(data_clean['BZ'][:3000], step=30)

hp.movable3D_hist(clean_BX_3D_0_3000_30)
hp.movable3D_hist(clean_BY_3D_0_3000_30)
hp.movable3D_hist(clean_BZ_3D_0_3000_30)

# ЕМ-алгоритм

* __Е-этап__
    1. Calculate unnormalized responsibilities: 
    $$
    \normalsize{ \quad \tilde\rho_k^{[i]} = \pi_k \cdot \frac{1}{\sigma_k \sqrt{2\pi}} \cdot \exp{\left(-\frac{(x^{[i]} - \mu_k)^2}{2\sigma^2}\right)} \equiv
    \pi_k \cdot \frac{1}{\sigma_k} \cdot \varphi \left(\frac{x^{[i]} - \mu_k}{2\sigma} \right) }
    $$
    2. Normilize responsibilities: 
    $$
    \normalsize{ \quad \rho_k^{[i]} = \frac{\tilde\rho_k^{[i]}}{\sum_{k=0}^{M-1} \tilde\rho_k^{[i]}} }
    $$
    3. Calculate class responsibilities: 
    $$
    \normalsize{ \quad \gamma_k = \sum_{i=0}^{N-1} \rho_k^{[i]} }
    $$
* __М-этап__
    1. Update the class probabilities: 
    $$
    \normalsize{ \quad \pi_k = \frac{\quad \gamma_k}{N} }
    $$
    2. Update the math. expectations: 
    $$
    \normalsize{ \quad \mu_k = \frac{1}{\gamma_k} \cdot \sum_{i=0}^{N-1} \rho_k^{[i]}x^{[i]} }
    $$
    3. Update the standard deviations:
    $$
    \normalsize{ \quad \sigma_k = \sqrt{\frac{1}{\gamma_k} \cdot \sum_{i=0}^{N-1} \rho_k^{[i]}\left(x^{[i]} - \mu_k \right)^2} }
    $$

## Реализация алгоритма

### Код для проверки корректности работы реализованного ЕМ-алгоритма

## Применение к склеиным данным.

In [ ]:
random_seed = 42 # for reproducability
n_iterations = 42
n_classes = 6
clean_mix_dicts = []
for n_classes in range(1,11):
    class_probs, mus, sigmas = es.EM(data_clean['BX'].values,
                                     n_classes, 
                                     n_iterations,
                                     random_seed)
    param = {}
    param.update({"class_probs": class_probs,
                  "mus" : mus,
                  "sigmas" : sigmas})
    clean_mix_dicts.append(param)
    print(f"Вероятности классов: {class_probs}")
    print(f"Мат. ожидания: {mus}")
    print(f"Ср. кв. отклонение: {sigmas}\n")

In [12]:
import silence_tensorflow.auto
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.special import logsumexp
import matplotlib.pyplot as plt

from tqdm import tqdm # progress-meter

class_probs_true = [0.6, 0.4]
mus_true = [2.5, 4.8]
sigmas_true = [0.6, 0.3]

random_seed = 42 # for reproducability
n_samples = 10000

n_iterations_pre_sieving = 5#**(-4) #epsilon
n_candidates = 100
n_iterations_post_sieving = 100#**(-2)
n_chosen_ones = 5
n_classes = 2

# generate the data
univariate_gmm = tfp.distributions.MixtureSameFamily(
    mixture_distribution=tfp.distributions.Categorical(probs=class_probs_true),
    components_distribution=tfp.distributions.Normal(
        loc=mus_true,
        scale=sigmas_true,
    )
)

dataset = univariate_gmm.sample(n_samples, seed=random_seed).numpy()

# for i in range(10):
dataset = univariate_gmm.sample(n_samples, seed=random_seed).numpy()
class_probs, mus, sigmas, log_likelihood_histories = es.em_sieved(
    dataset,
    n_classes,
    n_iterations_pre_sieving,
    n_candidates,
    n_iterations_post_sieving,
    n_chosen_ones,
    random_seed,
) 

print(class_probs)
print(mus)
print(sigmas)
print("---")

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 326.26it/s]


5 	 [0.49669923 0.50330077] 




100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 313.17it/s]


5 	 [2.05483644e-09 9.99999998e-01] 




100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 326.11it/s]


...

100 	 [0.59488446 0.40511554] 




100%|████████████████████████████████████████| 100/100 [00:00<00:00, 343.10it/s]


100 	 [0.59488446 0.40511554] 




100%|████████████████████████████████████████| 100/100 [00:00<00:00, 341.96it/s]

100 	 [0.59488446 0.40511554] 


[0.40511554 0.59488446]
[4.8054906  2.51388581]
[0.29521281 0.59199797]
---


In [ ]:
for x, y in zip([0.57106892, 0.42893108] ,[0.57106891, 0.42893109]):
    print(round(x-y, 9))

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
for log_likelihood_history in log_likelihood_histories:
    plt.plot(log_likelihood_history)
plt.xlabel("Iteration")
plt.ylabel("Log Likelihood")
plt.show()

In [11]:
random_seed = 142 # for reproducability

n_iterations_pre_sieving = 10 # Пороговое значение разности приближений на соседних итерациях. Крит-рий останова
n_candidates = 100
n_iterations_post_sieving = 200 # Кол-во итераций ЕМ для конкретного набора
n_chosen_ones = 3
n_classes = 3 # Кол-во выделяемых параметров смеси

class_probs, mus, sigmas, log_likelihood_histories = es.em_sieved(
    data_clean['BX'].values,
    n_classes,
    n_iterations_pre_sieving,
    n_candidates,
    n_iterations_post_sieving,
    n_chosen_ones,
    random_seed,
) 

print(class_probs)
print(mus)
print(sigmas)



100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 50.30it/s]


10 	 [0.02952343 0.00356142 0.00637382 0.45266703 0.50524834 0.00262596] 




100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 46.36it/s]


10 	 [1.95635332e-05 1.99770591e-05 2.71072036e-04 1.69962429e-05
 9.99502088e-01 1.70302946e-04] 




100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 49.14it/s]


...

10 	 [9.57115478e-05 2.35934658e-02 2.93475477e-03 9.72661789e-01
 3.31948520e-04 3.82330268e-04] 




100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 57.19it/s]


10 	 [8.25679990e-01 9.32799135e-03 3.10412180e-02 3.17138464e-02
 1.02048546e-01 1.88408934e-04] 




 90%|███████████████████████████████████████▌    | 9/10 [00:00<00:00, 49.10it/s]


KeyboardInterrupt: 

In [ ]:
hp.static2D_mixture(clean_mix_dicts, data_clean['BX'], mode='subplots')

hp.static2D_mixture(clean_mix_dicts, data_clean['BX'])

In [ ]:
import matplotlib.pyplot as plt
llhs = [es.log_likelihood(mixture3, series.values),
        es.log_likelihood(mixture6, series.values)]
colors = ['r','b']
for l, color in zip(llhs,colors):
    plt.plot(l,color=color)
plt.title(f"Точности: (1) для отбора - {conv_initial}\n(2) для лучших наборов - {conv_prime}")

In [ ]:
series = data_clean['BX'][:4000]
conv_prime=0.00001000
conv_initial=0.000100
window = 3000

mixture3 = es.mixture_extraction(series,
                            n_class=3,
                            conv_prime=conv_prime,
                            conv_initial=conv_initial,
                            rseed=45,
                            step=50,
                            window_size=window,
                            xaxis_id=data_clean['ydhm_id'],
                            EM_prog_bar=False)

# mixture6 = es.mixture_extraction(series,
#                             n_class=6,
#                             conv_prime=conv_prime,
#                             conv_initial=conv_initial,
#                             rseed=46,
#                             step=50,
#                             window_size=window,
#                             xaxis_id=data_clean['ydhm_id'],
#                             EM_prog_bar=False)

In [ ]:
mixture3["LL_hist"].plot()

In [ ]:
mixture3["LL_hist"].plot()

In [ ]:
mixture = es.mixture_extraction(data_clean['BX'],
                            n_class=3,
                            conv_prime=conv_prime*100,
                            conv_initial=conv_initial*10,
                            rseed=24012000,
                            step=10,
                            window_size=4000,
                            xaxis_id=data_clean['ydhm_id'],
                            EM_prog_bar=False)

In [ ]:
hp.construct_mixture_2Dplot(mixture)#.write_html(f"/home/oplora/Desktop/mix3.html")

In [ ]:
WS = 4500
EPS = 0.1
STEP = 10
SEED = 42

In [ ]:
param_df.loc[:,'lpl']

In [ ]:
BX_mixtures3 = es.mixture_extraction(data_clean['BX'], n_class=3, conv_initial=EPS, rseed=SEED,
                            step=STEP, window_size=WS, xaxis_id=data_clean['ydhm_id'], EM_prog_bar=True)
plot3 = hp.construct_mixture_2Dplot(BX_mixtures3)
plot3

In [ ]:
BX_mixtures6 = es.mixture_param(data_clean['BX'], n_class=6, eps=EPS,
                            step=STEP, window_size=WS, xaxis_id=data_clean['ydhm_id'])
plot6 = hp.construct_mixture_2Dplot(BX_mixtures6)
plot6

In [ ]:
name = '_seed'+f'{SEED}_eps{EPS}_w{WS}_st{STEP}'
BX_mixtures3.to_csv(PATH_DATA+f'bestBX_3c{name}.csv', index=False)
plot3.write_html(f"/home/oplora/Desktop/3c{name}.html")
#plot3.write_html(f"/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/3 comp/3c{name}.html")

In [ ]:
name = '_seed'+'27121954'
BX_mixtures6.to_csv(PATH_DATA+f'BX_6c{name}.csv', index=False)
plot6.write_html(f"/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/6 comp/6c{name}.html")

# Черновики

In [ ]:
len(class_prob_values)

In [ ]:
np.array(indexes[0])
#st_dev_values[:,i][0]

In [ ]:
st_dev_values[:,i]

In [ ]:
BX_mixtures6.columns

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
# plotly.offline.init_notebook_mode()

# Предположим, у нас есть DataFrame df с данными
# Ваш DataFrame с мульти-индексными колонками
df = BX_mixtures6

# Подготовка данных для графика
# Получение значений math_exp, st_dev, индексов и времени из DataFrame
math_exp_values = df['math_exp'].values
st_dev_values = df['st_dev'].values
indexes = df.index
indexes = df.attrs.get('custom_xaxis')
class_prob_values = df['cl_prob'].values

# Создаем график
fig = go.Figure()


# Добавляем линии для каждого набора значений low1, low2, low3
for i in range(6):
    fig.add_trace(go.Scatter3d(
        x=indexes, 
        y=st_dev_values[:,i], 
        z=math_exp_values[:,i],
        mode='lines',
        line=dict(color=class_prob_values[:,i],
                  colorscale=[(0, "#86001A"), (1, '#00FF00')],
                  colorbar=dict(title='Вес',len=0.75),
                  cmin=0, cmax=1
                 ),
        name=f'закон{i+1}',
        # Добавляем информацию о значении цвета
        text=[f'вес: {round(c, 5)}' for c in class_prob_values[:,i]],
        hoverlabel_font_color='blue') # текст слева при наведении курсора
                 )
    
    fig.add_trace(go.Scatter3d(
        x=np.array(indexes[0]), 
        y=st_dev_values[:1,i], 
        z=math_exp_values[:1,i],
        mode='markers',
        name=f'FFFF{i+1}')
        )
    
# Персонализация осей
    custom_scene = dict(
                xaxis = dict(
                    title='Дата',
                    color='grey'),
                yaxis = dict(
                    title='Среднекв-е откл-е, нТ$',
                    color='grey'),
                zaxis = dict(
                    title = "Мат-е ож-е, нТ",
                    color = 'grey'))
# Настройка макета графика
fig.update_layout(
    scene=custom_scene,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    title='3D График параметров смеси с цветовой шкалой для значения весов',
    coloraxis_colorbar=dict(
        x=-1.1,  # Расположение по оси X
        y=1.0   # Расположение по оси Y
    ),
    width=1000, 
    height=800,
    autosize=False
)

# Отображаем график
fig.show()

In [ ]:
df2 = pd.read_csv(PATH_DATA + "BX_3c.csv", dtype=TYPES)